In [242]:
import param as pm
import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv

In [285]:
b.param['mint'].crop_to_bounds(2)

-1

In [346]:
class BondingCurve(pm.Parameterized):
    supply = pm.Number(80, softbounds=(1, 100), bounds=(1, None), step=1)
    price = pm.Number(2, softbounds=(0.01, 5), bounds=(0.01, None),  step=0.01)
    balance = pm.Number(40, softbounds=(10, 500), bounds=(10, None), step=1)
    marketcap = pm.Number(4, constant=True)
    reserve_ratio = pm.Number(1, constant=True, step=0.01)
    n = pm.Number(constant=True)
    m = pm.Number(constant=True)
    
    mint = pm.Number(1, step=0.1)
    deposit = pm.Number(constant=True)
    mint_price = pm.Number(constant=True)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.update()
    
    @pm.depends('balance', 'supply', 'price', 'mint', watch=True)
    def update(self):
        with pm.edit_constant(self):
            self.marketcap = self.price * self.supply
            self.reserve_ratio = self.balance / self.marketcap
            self.param['mint'].bounds = [-self.supply+1, self.param['supply'].softbounds[1]-self.supply]
            # self.mint = self.param['mint'].crop_to_bounds(self.mint)
            self.n = ((1 / self.reserve_ratio) - 1)
            self.m = self.price / self.supply ** self.n

    def get_price(self, supply):
        price = self.m * supply ** self.n
        return price
    
    def get_marketcap(self, supply):
        marketcap = self.get_price(supply) * supply
        return marketcap
    
    def get_balance(self, supply):
        balance = self.reserve_ratio * self.get_marketcap(supply)
        return balance
        
    def price_over_supply_curve(self):
        supply = np.linspace(*self.param['supply'].softbounds, num=1000)
        prices = self.get_price(supply)
        balance_slope = np.where(supply <= self.supply, prices, 0)
        marketcap_slope = np.where(supply <= self.supply, self.price, 0)
        df = pd.DataFrame({
            'Supply': supply, 
            'Price': prices, 
            'Balance': balance_slope,
            'Marketcap': marketcap_slope,
        })
        return df
    
    def view_price_over_supply_curve(self):
        price_over_supply_curve = self.price_over_supply_curve()
        price_curve = price_over_supply_curve.hvplot.line(x='Supply',y='Price')
        price_curve.opts( 
            color='purple', 
        )
        balance_integral = price_over_supply_curve.hvplot.area(
            x='Supply', 
            y=['Marketcap', 'Balance'], 
            color=['green', 'blue'],
            alpha=0.4,
            stacked=False,
        )

        chart = price_curve * balance_integral
        return chart
    
    def view_points(self):
        current_price = (self.supply, self.price, 'Current Price')
        future_price = (self.supply+self.mint, self.get_price(self.supply+self.mint), 'Future Price')
        points = pd.DataFrame([future_price, current_price], columns=['x','y','label']).hvplot.scatter(
            x='x',
            y='y',
            by='label',
            color=['purple', 'orange'],
            size=80,) 
        return points
    
    @pm.depends('balance', 'supply', 'price', 'mint')
    def view_chart(self):
        curve = self.view_price_over_supply_curve()
        points = self.view_points()
        chart = curve * points
        chart.opts(
            title="Bonding Curve Math",
            legend_position="top_right",
            xlim=self.param['supply'].softbounds,
            ylim=self.param['price'].softbounds,
            width=420,
            height=620,
        )
        return chart
    
    def view(self):
        return pn.Row(self, self.view_chart)

In [347]:
b = BondingCurve()

In [348]:
b.view()

Row
    [0] Column(margin=5, name='BondingCurve', width=300)
        [0] StaticText(value='<b>BondingCurve</b>')
        [1] FloatSlider(end=100, name='Supply', start=1, step=1, value=80)
        [2] FloatSlider(end=5, name='Price', start=0.01, step=0.01, value=2)
        [3] FloatSlider(end=500, name='Balance', start=10, step=1, value=40)
        [4] FloatInput(disabled=True, name='Marketcap', value=160)
        [5] FloatInput(disabled=True, name='Reserve ratio', step=0.01, value=0.25)
        [6] FloatInput(disabled=True, name='N', value=3.0)
        [7] FloatInput(disabled=True, name='M', value=3.90625e-06)
        [8] FloatSlider(end=20, name='Mint', start=-79, value=1)
        [9] FloatInput(disabled=True, name='Deposit')
        [10] FloatInput(disabled=True, name='Mint price')
    [1] ParamMethod(method, _pane=HoloViews)

In [115]:
b.price_over_supply_curve()

,Supply,Price,Balance Slope
0,1.000000,3.890620,3.890620
1,1.009009,3.857266,3.857266
2,1.018018,3.824490,3.824490
3,1.027027,3.792278,3.792278
4,1.036036,3.760615,3.760615
...,...,...,...
995,9.963964,0.428079,0.000000
996,9.972973,0.427708,0.000000
997,9.981982,0.427337,0.000000
998,9.990991,0.426967,0.000000


In [25]:
class Exponential(pm.Parameterized):
    x = pm.Magnitude(0.5, softbounds=(0.01,1), step=0.01)
    m = pm.Number(0.5, bounds=(0,2))
    n = pm.Number(1, bounds=(0,2))
    
    def f(self, x):
        return self.m * x ** self.n
        
    def curve(self):
        xs = np.linspace(*self.param['x'].softbounds, num=1000)
        y = [self.f(x) for x in xs]
        return pd.DataFrame({'x':xs,'y':y})
    
    def view_x_point(self):
        return hv.Points((self.x, self.f(self.x))).opts(color='k', size=10)
    
    def view_curve(self):
        return self.curve().hvplot.line(x='x',y='y', xlim=(0,2), ylim=(0,2)) * self.view_x_point()
    
    def view_x_value(self):
        return pn.Column('#### Selected Value', f'({round(self.x,2)}, {round(self.f(self.x),2)})')
    
    def view(self):
        return pn.Row(pn.Column(self, self.view_x_value), self.view_curve)

e = Exponential()

e.view()

Row
    [0] Column
        [0] Column(margin=5, name='Exponential', width=300)
            [0] StaticText(value='<b>Exponential</b>')
            [1] FloatSlider(name='X', start=0.01, step=0.01, value=0.5)
            [2] FloatSlider(end=2, name='M', value=0.5)
            [3] FloatSlider(end=2, name='N', value=1)
        [1] ParamMethod(method, _pane=Column)
    [1] ParamMethod(method, _pane=HoloViews)

In [28]:
class LogExponential(Exponential):
    
    def f(self, x):
        return np.log(self.m) + self.n * np.log(x)
    
    def view_curve(self):
        return self.curve().hvplot.line(x='x',y='y') * self.view_x_point()
    
l = LogExponential()

l.view()

Row
    [0] Column
        [0] Column(margin=5, name='LogExponential', width=300)
            [0] StaticText(value='<b>LogExponential</b>')
            [1] FloatSlider(name='X', start=0.01, step=0.01, value=0.5)
            [2] FloatSlider(end=2, name='M', value=0.5)
            [3] FloatSlider(end=2, name='N', value=1)
        [1] ParamMethod(method, _pane=Column)
    [1] ParamMethod(method, _pane=HoloViews)